<a href="https://colab.research.google.com/github/karino2/US-patent-analysis/blob/bert_input/colab/bert_benchmark_2000_save_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BERT on 2000-2000 prediction benchmark, save partial result**

Benchmark 2000 seems too big to run on colab at once.
So I create notebook to predict 20 application each.

Basic code is from here: 
https://github.com/karino2/US-patent-analysis/blob/bert_input/colab/bert_benchmark_2000.ipynb

In [0]:
import os
import datetime
import pickle
import gzip

import tensorflow as tf

In [2]:
tf.__version__

'1.12.0'

In [3]:
!git clone -b docker https://github.com/yoheikikuta/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 174.52 KiB | 2.42 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [4]:
!ls

bert  sample_data


# Data setup

Data setup is based on https://github.com/karino2/US-patent-analysis/blob/bert_input/colab/bert_tokenized_claim_setup.ipynb

In [0]:
!mkdir ./bert/data

In [0]:
from google.colab import auth
auth.authenticate_user()

In [11]:
!gsutil cp gs://karino2-uspatent/features/test_grants_ids.pkl.gz ./bert/data/

Copying gs://karino2-uspatent/features/test_grants_ids.pkl.gz...
/ [1 files][  3.0 MiB/  3.0 MiB]                                                
Operation completed over 1 objects/3.0 MiB.                                      


In [12]:
!ls ./bert/data

test_grants_ids.pkl.gz


In [0]:
with gzip.open("./bert/data/test_grants_ids.pkl.gz", 'rb') as f:
     (test_ids, grants_ids) = pickle.load(f)

**Check tpu name**

In [14]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.94.132.42:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5251255233011251342),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12361401316273377857),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 9410561939850398054),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 190157728224345878),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13615944224243454595),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10869543044736146137),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 8495997812826624904),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 5036034131182578565),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 137633

### Bert setup

In [0]:
VOCAB_CONFIG_PATH=' gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12' #@param {type:"string"}

In [16]:
!gsutil cp {VOCAB_CONFIG_PATH}/vocab.txt ./bert/model/patent/vocab.txt
!gsutil cp {VOCAB_CONFIG_PATH}/bert_config.json ./bert/model/patent/bert_config.json

Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/vocab.txt...
/ [1 files][226.1 KiB/226.1 KiB]                                                
Operation completed over 1 objects/226.1 KiB.                                    
Copying gs://yohei-kikuta/mlstudy-phys/bert/models/pre-trained-models/uncased_L-12_H-768_A-12/bert_config.json...
/ [1 files][  313.0 B/  313.0 B]                                                
Operation completed over 1 objects/313.0 B.                                      


In [17]:
!ls ./bert/model/patent

bert_config.json  vocab.txt


In [0]:
import sys
sys.path.append("./bert")

In [0]:
import tokenization

In [0]:
tokenizer = tokenization.FullTokenizer(
    vocab_file="./bert/model/patent/vocab.txt", do_lower_case=True)

In [0]:
CLS_ID = tokenizer.vocab["[CLS]"]
SEP_ID = tokenizer.vocab["[SEP]"]

In [0]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

In [0]:
label_list = ["not_entailment", "entailment"]
max_seq_length = 512

label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i

In [0]:
def original_convert_single_pair(ids_a_input, ids_b_input):

  # Clone and pop for truncate. Most of the case result len is the same for our purpose, but try simple implementation first.
  ids_a = list(ids_a_input)
  ids_b = list(ids_b_input)
    
  # Modifies `tokens_a` and `tokens_b` in place so that the total
  # length is less than the specified length.
  # Account for [CLS], [SEP], [SEP] with "- 3"
  _truncate_seq_pair(ids_a, ids_b, max_seq_length - 3)

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  input_ids = []
  segment_ids = []
  input_ids.append(CLS_ID)
  segment_ids.append(0)
  for token in ids_a:
    input_ids.append(token)
    segment_ids.append(0)
  input_ids.append(SEP_ID)
  segment_ids.append(0)

  for token in ids_b:
    input_ids.append(token)
    segment_ids.append(1)
  input_ids.append(SEP_ID)
  segment_ids.append(1)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  return (input_ids, input_mask, segment_ids)


### Optimization for our application

dup ids consume huge memory.
We cach and use the same memory if ids_a_nput and ids_b_input have enough size.

In [0]:
NORMAL_INPUT_MASK = [1]*max_seq_length
NORMAL_SEGMENT_IDS = [0]*257+[1]*255

In [0]:
# [CLS](0), 1-255(len=255), [SEP], 257-510(len=254), [SEP]

In [0]:
LEN_255_CACHE = {}
LEN_254_CACHE = {}

In [0]:
def truncage_with_cache(ids, target_len):
  if target_len == 255:
    cache = LEN_255_CACHE
  else:
    assert target_len == 254
    cache = LEN_254_CACHE
  if ids not in cache:
    cache[ids] = ids[0:target_len]
  return cache[ids]

In [0]:
def fast_convert_single_pair(ids_a_input, ids_b_input):  
  if (len(ids_a_input) < 256) or (len(ids_b_input) < 256) :
    return original_convert_single_pair(ids_a_input, ids_b_input)
  
  ids_a = truncage_with_cache(ids_a_input, 255)
  ids_b = truncage_with_cache(ids_b_input, 254)

  input_ids = (CLS_ID,) + ids_a + (SEP_ID,) + ids_b + (SEP_ID,)
  
  return (input_ids, NORMAL_INPUT_MASK, NORMAL_SEGMENT_IDS)


In [0]:
def feature_tuplist_to_feature_dict(ftups):
    dic = {}
    dic['input_ids'] = [tup[0] for tup in ftups]
    dic["input_mask"] = [tup[1] for tup in ftups]
    dic["segment_ids"] = [tup[2] for tup in ftups]
    dic["label_ids"] = [[0] for _ in ftups]
    return dic

List is not hashable, but all element is int and tuple is hashable in this case. We use this ids as key for caching.

In [0]:
test_ids_tup = [tuple(ids) for ids in test_ids]

In [0]:
grants_ids_tup = [tuple(ids) for ids in grants_ids]

In [0]:
pairs = [("{}_{}".format(i, j), app_claim, grants_claim) for i, app_claim in enumerate(test_ids_tup) for j, grants_claim in enumerate(grants_ids_tup)]

In [0]:
import os

import modeling
import tokenization

from run_classifier import model_fn_builder


In [0]:
bert_config = modeling.BertConfig.from_json_file("./bert/model/patent/bert_config.json")

In [36]:
BUCKET = 'karino2-uspatent' #@param {type:"string"}
OUTPUT_DIR = 'gs://{}/mlstudy-phys/bert/models/patent_1000'.format(BUCKET)

INIT_CHECK_POINT_FILE='model.ckpt-1602' #@param {type:"string"}
INIT_CHECK_POINT = '{}/{}'.format(OUTPUT_DIR, INIT_CHECK_POINT_FILE)

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))
print('***** Init checkpoint: {} *****'.format(INIT_CHECK_POINT))

***** Model output directory: gs://karino2-uspatent/mlstudy-phys/bert/models/patent_1000 *****
***** Init checkpoint: gs://karino2-uspatent/mlstudy-phys/bert/models/patent_1000/model.ckpt-1602 *****


In [0]:
class FLAGS(object):
    '''Parameters.'''
    def __init__(self):
        self.vocab_file = "./bert/model/patent/vocab.txt"
        self.use_tpu = True
        self.output_dir = OUTPUT_DIR
        # self.data_dir = "./bert/data/1000"
        self.init_checkpoint = INIT_CHECK_POINT
        self.predict_batch_size = 16
        # TPU related
        self.num_tpu_cores = 8
        self.tpu_name = TPU_ADDRESS
        
        # following parameters are not used anymore. (because we create feature by hand)
        self.do_lower_case = True
        self.max_seq_length = 512
        
        # The following parameters are not used in predictions.
        # Just use to create RunConfig.
        self.master = None
        self.save_checkpoints_steps = 1
        self.learning_rate = 0
        self.num_warmup_steps = 0
        self.num_train_steps = 0
        # TPU related. Some of these value have positive int not to make TPUEstimator angry (even though these are not used...).
        self.eval_batch_size = 32
        self.train_batch_size = 32 
        self.iterations_per_loop = 1000


In [0]:
FLAGS = FLAGS()

In [0]:
# tokenizer = tokenization.FullTokenizer(
#    vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

# tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=FLAGS.tpu_name).get_master()
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu=FLAGS.tpu_name)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

In [0]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(["not_entailment", "entailment"]),
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=FLAGS.num_train_steps,
    num_warmup_steps=FLAGS.num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_tpu)

In [0]:
# https://github.com/kyzhouhzau/BERT-NER/issues/19

def model_fn_wrapper(features, labels, mode, params):
  tspec = model_fn(features, labels, mode, params)
  pred_dict = {'predictions': tspec.predictions}
  return tf.contrib.tpu.TPUEstimatorSpec(
          mode=tspec.mode, predictions=pred_dict, scaffold_fn=tspec.scaffold_fn)


In [42]:
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    # model_fn=model_fn,
    model_fn=model_fn_wrapper,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

INFO:tensorflow:Using config: {'_model_dir': 'gs://karino2-uspatent/mlstudy-phys/bert/models/patent_1000', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      value: "10.94.132.42:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f18783b09b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'grpc://10.94.132.42:8470', '_evaluation_master': b'grpc://10.94.132.42:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, nu

### Whole data set is too big for memory

Specify which block we will handle.

In [43]:
WHOLE_APP_NUM=1000

WHOLE_PAIR_NUM = len(pairs)
WHOLE_GRANTS_NUM = WHOLE_PAIR_NUM//WHOLE_APP_NUM
WHOLE_PAIR_NUM, WHOLE_GRANTS_NUM

(2524000, 2524)

In [0]:
BLOCK_SIZE = 20*WHOLE_GRANTS_NUM

In [45]:
TOTAL_BLOCK_NUM = WHOLE_PAIR_NUM//BLOCK_SIZE
BLOCK_SIZE, TOTAL_BLOCK_NUM

(50480, 50)

In [0]:
def predict_pairs(pair_begin, pair_end):
  fdict = feature_tuplist_to_feature_dict([fast_convert_single_pair(tup[1], tup[2]) for tup in pairs[pair_begin:pair_end]])
  
  def _batch_input_function(params):
    test_ds = tf.data.Dataset.from_tensor_slices(fdict)    
    return test_ds.batch(params['batch_size'])
  
  result = estimator.predict(_batch_input_function)
  result = list(result)
  print(datetime.datetime.today())
  
  return  [pred['predictions'] for pred in result]

def predict_block_and_save_result(block_id):
  pair_begin = (block_id)*BLOCK_SIZE
  pair_end = (block_id+1)*BLOCK_SIZE
  print("{}: start predict {}-{}".format(datetime.datetime.today(), pair_begin, pair_end))

  result_unwrap = predict_pairs(pair_begin, pair_end)
  
  filename = "result_{}_{}.pkl.gz".format(pair_begin, pair_end)
  
  with gzip.open(filename, 'w') as f:
     pickle.dump(result_unwrap, f)
  !gsutil cp {filename} gs://karino2-uspatent/features/

Logging takes very long time (it takes 42min for 5min task.)
Disable it.

In [0]:
tf.logging.set_verbosity(tf.logging.ERROR)

Block_id is from 0 to 49.

Cell below takes about 4min.

In [57]:
#block_id 0 to 49
# predict_block_and_save_result(0)
# predict_block_and_save_result(1)
# predict_block_and_save_result(2)

predict_block_and_save_result(3)

2019-01-09 04:29:50.068956: start predict 151440-201920
2019-01-09 04:33:41.288760
Copying file://result_151440_201920.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      


In [59]:
predict_block_and_save_result(4)
predict_block_and_save_result(5)
predict_block_and_save_result(6)
predict_block_and_save_result(7)
predict_block_and_save_result(8)
predict_block_and_save_result(9)

2019-01-09 04:36:16.297004: start predict 201920-252400
2019-01-09 04:40:08.073022
Copying file://result_201920_252400.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
2019-01-09 04:40:12.637301: start predict 252400-302880
2019-01-09 04:44:02.763365
Copying file://result_252400_302880.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
2019-01-09 04:44:08.222916: start predict 302880-353360
2019-01-09 04:48:01.175177
Copying file://result_302880_353360.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
2019

In [60]:
for i in range(10, 20):
  predict_block_and_save_result(i)

2019-01-09 05:02:12.300972: start predict 504800-555280
2019-01-09 05:06:03.637383
Copying file://result_504800_555280.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
2019-01-09 05:06:09.139959: start predict 555280-605760
2019-01-09 05:10:00.807352
Copying file://result_555280_605760.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
2019-01-09 05:10:05.610673: start predict 605760-656240
2019-01-09 05:13:56.416769
Copying file://result_605760_656240.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
2019

In [61]:
for i in range(20, 30):
  predict_block_and_save_result(i)

2019-01-09 05:44:01.064543: start predict 1009600-1060080
2019-01-09 05:47:52.602842
Copying file://result_1009600_1060080.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
2019-01-09 05:47:56.884997: start predict 1060080-1110560
2019-01-09 05:51:47.124555
Copying file://result_1060080_1110560.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
2019-01-09 05:51:52.543860: start predict 1110560-1161040
2019-01-09 05:55:44.712423
Copying file://result_1110560_1161040.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                               

In [62]:
for i in range(30, 35):
  print("block_id: {}".format(i))
  predict_block_and_save_result(i)

block_id: 30
2019-01-09 06:43:44.801708: start predict 1514400-1564880
2019-01-09 06:47:36.979390
Copying file://result_1514400_1564880.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
block_id: 31
2019-01-09 06:47:42.700069: start predict 1564880-1615360
2019-01-09 06:51:33.107563
Copying file://result_1564880_1615360.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
block_id: 32
2019-01-09 06:51:37.686808: start predict 1615360-1665840
2019-01-09 06:55:28.366338
Copying file://result_1615360_1665840.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/

In [48]:
for i in range(35, 45):
  print("block_id: {}".format(i))
  predict_block_and_save_result(i)

block_id: 35
2019-01-09 11:00:21.430526: start predict 1766800-1817280
2019-01-09 11:04:27.189020
Copying file://result_1766800_1817280.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
block_id: 36
2019-01-09 11:04:31.693844: start predict 1817280-1867760
2019-01-09 11:08:22.101062
Copying file://result_1817280_1867760.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][ 1022 KiB/ 1022 KiB]                                                
Operation completed over 1 objects/1022.5 KiB.                                   
block_id: 37
2019-01-09 11:08:27.644275: start predict 1867760-1918240
2019-01-09 11:12:17.713533
Copying file://result_1867760_1918240.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/

In [49]:
for i in range(45, 50):
  print("block_id: {}".format(i))
  predict_block_and_save_result(i)

block_id: 45
2019-01-09 11:42:04.541820: start predict 2271600-2322080
2019-01-09 11:45:54.839242
Copying file://result_2271600_2322080.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      
block_id: 46
2019-01-09 11:46:01.014163: start predict 2322080-2372560
2019-01-09 11:49:53.430954
Copying file://result_2322080_2372560.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][ 1024 KiB/ 1024 KiB]                                                
Operation completed over 1 objects/1024.0 KiB.                                   
block_id: 47
2019-01-09 11:49:58.034481: start predict 2372560-2423040
2019-01-09 11:53:49.030661
Copying file://result_2372560_2423040.pkl.gz [Content-Type=application/octet-stream]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/